# Imports

In [ ]:
import os, glob
import numpy as np
import pandas as pd

import math
import itertools

import matplotlib as mpl
import matplotlib.pyplot as plt

# Functions

In [ ]:
def style_plot():
    mpl.rcParams['lines.linewidth'] = .85
    plt.style.use('seaborn-whitegrid')
    plt.rcParams['figure.figsize'] = (30,15)

In [ ]:
def get_kadonis_df(filename):
    # open kadonis file
    f = open(filename,'r')
    # read lines and close
    lines = f.read().splitlines()
    f.close()
    # strip out lines we want
    lines_imp = lines[19:]
    x = [line.split() for line in lines_imp]
    # get column labels, column data
    headers = x[0]
    data = x[2:]
    # convert to df
    df = pd.DataFrame(data, columns=headers)
    # make numeric
    ene  = pd.to_numeric(df['E(c.m.)'])
    cs_true = pd.to_numeric(df['CS'])        
    cs_err  = pd.to_numeric(df['ErrCS'], errors='coerce')
    # check to make sure error bars are there
    if(math.isnan(cs_err[0])):
        cs_err = pd.Series([0 for x in cs_err])
        print('\n\n\nTHE DATA IN FILE {} DID NOT INCLUDE ERROR BARS.\n\n\n'.format(filename))
    # new df  
    df_new  = pd.DataFrame([ene, cs_true, cs_err]).transpose()
    df_new.columns = ['ene', 'cs_kadonis', 'cs_err']
    df_new = df_new.sort_values('ene', ascending=True)
    return(df_new)

In [ ]:
def get_talys_df(filename):
    f = open(filename,'r')
    # read lines and close
    lines = f.read().splitlines()
    f.close()
    # strip out lines we want
    lines_imp = lines[5:]
    x = [line.strip().split(' ') for line in lines_imp]
    df = pd.DataFrame(x)
    df.columns = ['ene','cs']
    # make numeric 
    df  = pd.DataFrame([pd.to_numeric(df['ene']),
                        # convert millibarns
                        pd.to_numeric(df['cs'])/1000]).transpose()
    return(df)

In [ ]:
### CHANGE THIS CELL ONLY IF YOU CHANGE THE NAME OF THE NOTEBOOK
def clean_files():
    # clean file system
    ### WHERE YOU SEE 'template.ipynb', ENTER THE NEW NAME OF THE NOTEBOOK
    keeping = [ag_talys_file, an_talys_file, 'output', 'total.tot', 'template.ipynb','input']
    files = [f for f in os.listdir('.') if os.path.isfile(f)]
    for f in files:
        if(f not in keeping):
            os.remove(f)
    print('Done --- File system clean.')

# Kadonis Portion

In [ ]:
### CHANGE THIS CELL
# read in files
ag_kad_file = './kad_files/XXXXXXXXXXXXXX'
an_kad_file = './kad_files/XXXXXXXXXXXXXX'

In [ ]:
# read in kadonis data into kad df's
ag_kad_df = get_kadonis_df(ag_kad_file)
an_kad_df = get_kadonis_df(an_kad_file)

In [ ]:
# check data in df
print(ag_kad_df)
print(an_kad_df)

## Alpha-Gamma Kadonis Plot

In [ ]:
style_plot()
plt.yscale('log')
plt.errorbar(ag_kad_df.ene, ag_kad_df['cs_kadonis'], yerr=ag_kad_df['cs_err'])

## Alpha-N Kadonis Plot

In [ ]:
style_plot()
plt.yscale('log')
plt.errorbar(an_kad_df.ene, an_kad_df['cs_kadonis'], yerr=an_kad_df['cs_err'])

# Talys Portion

In [ ]:
### CHANGE THIS CELL
# general parameters
symbol = 'xx'
mass = XXX
z = XXX

In [ ]:
# write energies file
ene_kad = []
[ene_kad.append(round(ene,4)) for ene in ag_kad_df['ene']]
[ene_kad.append(round(ene,4)) for ene in an_kad_df['ene']]
ene_kad = sorted(list(set(ene_kad)))
print(ene_kad)

f = open('energies','w')
[f.write(str(ene)+'\n') for ene in ene_kad]
f.close()

In [ ]:
# specify files to keep (still needs work)
if(z+2 < 100):
    pref = '0'+str(z+2)
else:
    pref = str(z+2)
ag_talys_file = 'rp'+pref+str(mass+4)+'.tot'
an_talys_file = 'rp'+pref+str(mass+3)+'.tot'

In [ ]:
# sanity check
print('Alpha-Gamma TALYS File:\n{}\nAlpha-N TALYS File:\n{}'.format(ag_talys_file, an_talys_file))

In [ ]:
# global models
ldmodel_glob = [1,2,3]
strength_glob = [1,2]
alphaomp_glob = [1,2,6,7,8]

# microscopic models
ldmodel_micro = [4,5,6]
strength_micro = [3,4,5,6,7,8]
alphaomp_micro = [3,4,5]

# all models
jlmomp = ['y','n']

In [ ]:
# make combinations that are allowed - forget the rest
global_params = [ldmodel_glob, strength_glob, alphaomp_glob, jlmomp]
micro_params  = [ldmodel_micro, strength_micro, alphaomp_micro, jlmomp]

global_combos = list(itertools.product(*global_params))
micro_combos  = list(itertools.product(*micro_params))

# all_combos = global_combos
all_combos = global_combos + micro_combos

In [ ]:
# start with energies in 'all' dataframe
ag_talys_all_df = pd.DataFrame(ene_kad)
an_talys_all_df = pd.DataFrame(ene_kad)
col_names = ['ene']

## Talys Loop

In [ ]:
# for tracking purposes
total_iter = len(all_combos)
current_iter = 1

for combo in all_combos:

    # get model parameters
    l = combo[0]
    s = combo[1]
    a = combo[2]
    j = combo[3]
    
    # make column name
    col_name = 'l'+str(l)+'s'+str(s)+'a'+str(a)+'j'+j
    col_names.append(col_name)

    # print progress
    print('({}/{}) --- Running {}...'.format(current_iter, total_iter, col_name))

    # write input file contents
    top = 'projectile a\nelement '+symbol+'\nmass '+str(mass)+'\nenergy energies\n'
    mid = 'ldmodel '+str(l)+'\nstrength '+str(s)+'\nalphaomp '+str(a)+'\njlmomp '+j+'\n'
    bottom1 = 'segment 3\nxseps 1.e-35\npopeps 1.e-35\ntranseps 1.e-35\n'
    bottom2 = 'cbreak p 0.\ncbreak n 0.\ncstrip p 0.\ncstrip n 0.\ncknock p 0.\ncknock n 0.\ngnorm 1'

    
    # actually write input file
    f = open('input', 'w')
    f.write(top+mid+bottom1+bottom2)
    f.close()

    # run talys
    !talys <input> output

    # concatenate current frame to "all" frame
    ag_talys_all_df = pd.concat([ag_talys_all_df, get_talys_df(ag_talys_file)['cs']], axis=1)
    an_talys_all_df = pd.concat([an_talys_all_df, get_talys_df(an_talys_file)['cs']], axis=1)

    # update iter count
    current_iter = current_iter + 1
         
# clean file system
clean_files()

In [ ]:
# rename columns
ag_talys_all_df.columns = col_names
an_talys_all_df.columns = col_names
print(ag_talys_all_df)
print(an_talys_all_df)

In [ ]:
an_talys_all_df.drop('ene',axis=1).plot(logy=True)

In [ ]:
# merge talys (all_df) with kadonis
# ag_mega_df = pd.concat([ag_kad_df, ag_talys_all_df],axis=1).dropna()
# an_mega_df = pd.concat([an_kad_df, an_talys_all_df],axis=1).dropna()
ag_mega_df = pd.concat([ag_kad_df, ag_talys_all_df],axis=1)
an_mega_df = pd.concat([an_kad_df, an_talys_all_df],axis=1)

In [ ]:
# strip duplicate ene column and set index to 'ene'
ag_mega_df = ag_mega_df.iloc[:,~ag_mega_df.columns.duplicated()].set_index('ene')
an_mega_df = an_mega_df.iloc[:,~an_mega_df.columns.duplicated()].set_index('ene')

In [ ]:
# sanity check
print(ag_mega_df)
print(an_mega_df)

# Plotting (all)

## Alpha-Gamma (kadonis + all talys)

In [ ]:
# plot styling
style_plot()

# plot data
ag_mega_df.drop(['cs_kadonis','cs_err'], axis=1).plot(logy=True, legend=None)
plt.errorbar(ag_mega_df.index, ag_mega_df['cs_kadonis'], yerr=ag_mega_df['cs_err'])

plt.title('Alpha-Gamma Reaction')
plt.xlabel('Energy (MeV)')
plt.ylabel('Cross Section (millibarns)')

## Alpha-N (kadonis + all talys)

In [ ]:
style_plot()

# plot data
an_mega_df.drop(['cs_kadonis','cs_err'], axis=1).plot(logy=True, legend=None)
plt.errorbar(an_mega_df.index, an_mega_df['cs_kadonis'], yerr=an_mega_df['cs_err'])

plt.title('Alpha-N Reaction')
plt.xlabel('Energy (MeV)')
plt.ylabel('Cross Section (millibarns)')

# Chi-Square Test (Goodness-of-Fit)

In [ ]:
def chi_square(observed, expected):
    return sum([((o-e)**2)/e for o, e in zip(observed,expected)])

In [ ]:
def chi_square_min(df):
    chi_square_values = []
    for col in df.drop(['cs_kadonis','cs_err'],axis=1).columns:
        chi_square_values.append(chi_square(df[col],df['cs_kadonis']))
    return(col_names[1:][chi_square_values.index(min(chi_square_values))])

# Plotting (best only)

## Alpha-Gamma (kadonis + best talys)

In [ ]:
style_plot()

print('Best Talys Combination: {}'.format(chi_square_min(ag_mega_df)))

# plot data
ag_mega_df[chi_square_min(ag_mega_df)].plot(logy=True)
plt.errorbar(ag_mega_df.index, ag_mega_df['cs_kadonis'], yerr=ag_mega_df['cs_err'])

plt.title('Alpha-Gamma Reaction')
plt.xlabel('Energy (MeV)')
plt.ylabel('Cross Section (millibarns)')

## Alpha-N (kadonis + best talys)

In [ ]:
style_plot()

print('Best Talys Combination: {}'.format(chi_square_min(an_mega_df)))

# plot data
an_mega_df[chi_square_min(an_mega_df)].plot(logy=True)
plt.errorbar(an_mega_df.index, an_mega_df['cs_kadonis'], yerr=an_mega_df['cs_err'])

plt.title('Alpha-N Reaction')
plt.xlabel('Energy (MeV)')
plt.ylabel('Cross Section (millibarns)')